In [40]:
import h2o

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 21.0.1+12-LTS-29, mixed mode, sharing)
  Starting server from C:\work\DrugDiscovery\drug-discovery-venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\38066\AppData\Local\Temp\tmp9ax3y1aq
  JVM stdout: C:\Users\38066\AppData\Local\Temp\tmp9ax3y1aq\h2o_38066_started_from_python.out
  JVM stderr: C:\Users\38066\AppData\Local\Temp\tmp9ax3y1aq\h2o_38066_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Kiev
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,3 months and 29 days
H2O_cluster_name:,H2O_from_python_38066_fvy2r0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.953 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


logP standart scaler

In [23]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\train_logp_standart_scaler.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\test_logp_standart_scaler.csv', index_col=0)

train = h2o.H2OFrame(train_df)
test = h2o.H2OFrame(test_df)

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [24]:
x = train.columns
x.remove('logP')
x.remove('fold_id')
y = 'logP'

In [25]:
from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_num_rules", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\train_logp_minmax.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\pKA_logP_minmaxscaler\logP\test_logp_minmax.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=20, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_train_cv, metrics_oos_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_oos_{cross_val_index}_mse"] = metrics_oos_cv.mse()
            cv_dict[f"cv_oos_{cross_val_index}_mae"] = metrics_oos_cv.mae()
            cv_dict[f"cv_oos_{cross_val_index}_r2"] = metrics_oos_cv.r2()
            cv_dict[f"cv_train_{cross_val_index}_mse"] = metrics_train_cv.mse()
            cv_dict[f"cv_train_{cross_val_index}_mae"] = metrics_train_cv.mae()
            cv_dict[f"cv_train_{cross_val_index}_r2"] = metrics_train_cv.r2()
            total_r_2_score += metrics_oos_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('logP')
        x.remove('fold_id')
        y = 'logP'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance_train_cv = rfit.model_performance(test_data=train)
        performance_oos_cv = rfit.model_performance(test_data=test)
        return performance_train_cv, performance_oos_cv


Trainer()

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
████████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
██████████████████████████████████████████

In [27]:
crossvals_list

[{'params': {'max_num_rules': 681, 'max_rule_length': 14, 'min_num_rules': 4},
  'cv_oos_0_mse': 0.08491880247328844,
  'cv_oos_0_mae': 0.24158989144813614,
  'cv_oos_0_r2': 0.7164159437299279,
  'cv_train_0_mse': 0.04209681813492891,
  'cv_train_0_mae': 0.15272321755253918,
  'cv_train_0_r2': 0.8747057276172305,
  'cv_oos_1_mse': 0.12003942156275012,
  'cv_oos_1_mae': 0.2621783010050563,
  'cv_oos_1_r2': 0.6427223565033768,
  'cv_train_1_mse': 0.028387064586367233,
  'cv_train_1_mae': 0.13561249395552558,
  'cv_train_1_r2': 0.9052021615173538},
 {'params': {'max_num_rules': 423, 'max_rule_length': 17, 'min_num_rules': 5},
  'cv_oos_0_mse': 0.08494652115772093,
  'cv_oos_0_mae': 0.241744965174752,
  'cv_oos_0_r2': 0.7163233779289875,
  'cv_train_0_mse': 0.0420983815581392,
  'cv_train_0_mae': 0.15269605438046754,
  'cv_train_0_r2': 0.8747010743445548,
  'cv_oos_1_mse': 0.12006288081253992,
  'cv_oos_1_mae': 0.26220177905393505,
  'cv_oos_1_r2': 0.6426525338953204,
  'cv_train_1_mse': 0

In [26]:
from h2o.estimators import H2ORuleFitEstimator

rfit = H2ORuleFitEstimator(
                           max_rule_length=18,
                           max_num_rules=118,
                           min_rule_length=5,
                           seed=1,
                           model_type="rules")
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |

██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.01525 ),52146,20,1,52146.0,700.0,700.0,0.0,18.0,9.837143,0.0,99.0,74.494286


In [28]:
performance = rfit.model_performance(test_data=test)

print(performance)

ModelMetricsRegressionGLM: rulefit
** Reported on test data. **

MSE: 0.09599259345599562
RMSE: 0.3098267152070583
MAE: 0.25826264973214735
RMSLE: 0.10089954660642002
Mean Residual Deviance: 0.09599259345599562
R^2: 0.6394756050461083
Null degrees of freedom: 24
Residual degrees of freedom: 4
Null deviance: 6.687705755427244
Residual deviance: 2.3998148363998903
AIC: 56.35982061046673


default logP

In [29]:
LOGP_FEATURES = ['f_freedom', 'PPSA5', 'mol_num_cycles', 'nFRing', 'nF', 'identificator',
                 'mol_weight', 'dipole_moment', 'nHRing', 'nO', 'PBF', 'nC', 'nARing',
                 'cis/trans', 'PNSA5', 'FPSA3', 'mol_volume', 'RPCS', 'GeomShapeIndex',
                 'WPSA5', 'TASA', 'f_to_fg', 'avg_atoms_in_cycle', 'nFHRing',
                 'chirality']


PKA_FEATURES = ['RPCS', 'PBF', 'mol_weight', 'dipole_moment', 'PPSA5',
                'avg_atoms_in_cycle', 'nHRing', 'cis/trans', 'FPSA3', 'nF', 'chirality',
                'sasa', 'PNSA5', 'GeomShapeIndex', 'TASA', 'mol_num_cycles',
                'f_freedom', 'nFRing', 'identificator', 'nO', 'nARing', 'nC', 'nFHRing',
                'f_to_fg']

In [30]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_logP_v4_features_2.2.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_logP_v4_features_2.2.csv', index_col=0)

train = h2o.H2OFrame(train_df[LOGP_FEATURES + ['logP', 'fold_id']])
test = h2o.H2OFrame(test_df[LOGP_FEATURES + ['logP']])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [31]:
x = train.columns
x.remove('logP')
x.remove('fold_id')
y = 'logP'

In [33]:
from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_num_rules", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_logP_v4_features_2.2.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_logP_v4_features_2.2.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=20, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_train_cv, metrics_oos_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_oos_{cross_val_index}_mse"] = metrics_oos_cv.mse()
            cv_dict[f"cv_oos_{cross_val_index}_mae"] = metrics_oos_cv.mae()
            cv_dict[f"cv_oos_{cross_val_index}_r2"] = metrics_oos_cv.r2()
            cv_dict[f"cv_train_{cross_val_index}_mse"] = metrics_train_cv.mse()
            cv_dict[f"cv_train_{cross_val_index}_mae"] = metrics_train_cv.mae()
            cv_dict[f"cv_train_{cross_val_index}_r2"] = metrics_train_cv.r2()
            total_r_2_score += metrics_oos_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('logP')
        x.remove('fold_id')
        y = 'logP'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance_train_cv = rfit.model_performance(test_data=train)
        performance_oos_cv = rfit.model_performance(test_data=test)
        return performance_train_cv, performance_oos_cv


Trainer()

NEW TRAIN STEP                                        
  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
█                                                     
███████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
█                                          

In [34]:
crossvals_list

[{'params': {'max_num_rules': 396, 'max_rule_length': 16, 'min_num_rules': 5},
  'cv_oos_0_mse': 0.07892161580526996,
  'cv_oos_0_mae': 0.22652308197497836,
  'cv_oos_0_r2': 0.7364433872641256,
  'cv_train_0_mse': 0.040501921432919426,
  'cv_train_0_mae': 0.14383405973918878,
  'cv_train_0_r2': 0.8794526759771625,
  'cv_oos_1_mse': 0.1422367766820041,
  'cv_oos_1_mae': 0.30772148126608406,
  'cv_oos_1_r2': 0.5766555708956249,
  'cv_train_1_mse': 0.028870187478788223,
  'cv_train_1_mae': 0.13891994966612128,
  'cv_train_1_r2': 0.9035887856156768},
 {'params': {'max_num_rules': 981, 'max_rule_length': 12, 'min_num_rules': 5},
  'cv_oos_0_mse': 0.07728107401571166,
  'cv_oos_0_mae': 0.22329249143275445,
  'cv_oos_0_r2': 0.7419219324344941,
  'cv_train_0_mse': 0.03979887407750023,
  'cv_train_0_mae': 0.14255663756360343,
  'cv_train_0_r2': 0.8815451810820742,
  'cv_oos_1_mse': 0.1422042595821618,
  'cv_oos_1_mae': 0.3077269354329835,
  'cv_oos_1_r2': 0.5767523527082474,
  'cv_train_1_mse':

In [35]:
from h2o.estimators import H2ORuleFitEstimator

rfit = H2ORuleFitEstimator(
                           max_rule_length=10,
                           max_num_rules=750,
                           min_rule_length=6,
                           seed=1,
                           model_type="rules")
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |

██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.009761 ),16648,45,1,16648.0,250.0,250.0,0.0,10.0,7.992,0.0,98.0,66.592


pKa default

In [41]:
from functools import partial

from h2o.estimators import H2ORuleFitEstimator
from hyperopt import fmin, tpe, hp

crossvals_list = []

class Trainer:
    def __init__(self) -> None:
        self.space = {
            # 'lr': hp.loguniform('lr', -10, -3),
            'max_rule_length': hp.randint("max_rule_length", 10, 20),
            'max_num_rules': hp.randint("max_num_rules", 1000),
            'min_num_rules': hp.randint("min_num_rules", 3, 10)
        }

        self.train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_pKa_v4_features_2.2.csv', index_col=0)
        self.test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_pKa_v4_features_2.2.csv', index_col=0)

        self.find_best_params_with_hyperopt()

    def find_best_params_with_hyperopt(self, ):
        algo = tpe.suggest

        objective_partial = partial(Trainer.optimization_function, X_train=self.train_df)

        best_hyperparams = fmin(fn=objective_partial, space=self.space, algo=algo, max_evals=20, verbose=1)

        print("Найкращі гіперпараметри:", best_hyperparams)
        return best_hyperparams

    @staticmethod
    def optimization_function(params, X_train):
        global crossvals_list

        amount_of_cross_vals = 2

        cv_indices_dict = {0: [], 1: []}
        for index, row in X_train.iterrows():
            cv_indices_dict[row['fold_id']].append(index)

        cv_indices = [[cv_indices_dict[0], cv_indices_dict[1]], [cv_indices_dict[1], cv_indices_dict[0]]]
        
        cv_dict = {"params": params}

        total_r_2_score = 0
        for cross_val_index in range(amount_of_cross_vals):
            # df.loc[df.index[index_list]]
            train_df_cv = X_train.loc[cv_indices[cross_val_index][0]]

            test_df_cv = X_train.loc[cv_indices[cross_val_index][1]]

            metrics_train_cv, metrics_oos_cv = Trainer.train(
                train_df_cv,
                test_df_cv,
                params['max_rule_length'],
                params['max_num_rules'],
                params['min_num_rules']
            )
            cv_dict[f"cv_oos_{cross_val_index}_mse"] = metrics_oos_cv.mse()
            cv_dict[f"cv_oos_{cross_val_index}_mae"] = metrics_oos_cv.mae()
            cv_dict[f"cv_oos_{cross_val_index}_r2"] = metrics_oos_cv.r2()
            cv_dict[f"cv_train_{cross_val_index}_mse"] = metrics_train_cv.mse()
            cv_dict[f"cv_train_{cross_val_index}_mae"] = metrics_train_cv.mae()
            cv_dict[f"cv_train_{cross_val_index}_r2"] = metrics_train_cv.r2()
            total_r_2_score += metrics_oos_cv.r2()

        crossvals_list.append(cv_dict)
        return -total_r_2_score
            
    @staticmethod
    def train(train_df, test_df,
              max_rule_length,
              max_num_rules,
              min_num_rules):
        print("NEW TRAIN STEP")
        train = h2o.H2OFrame(train_df)
        test = h2o.H2OFrame(test_df)

        x = train.columns
        x.remove('pKa')
        x.remove('fold_id')
        y = 'pKa'

        rfit = H2ORuleFitEstimator(
            max_rule_length=int(max_rule_length),
            max_num_rules=int(max_num_rules),
            # min_rule_length=int(min_num_rules),
            seed=1,
            model_type="rules"
        )
        rfit.train(training_frame=train, x=x, y=y)

        performance_train_cv = rfit.model_performance(test_data=train)
        performance_oos_cv = rfit.model_performance(test_data=test)
        return performance_train_cv, performance_oos_cv


Trainer()

NEW TRAIN STEP                                        
  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
█                                                     
█                                                     
███████████████████████████████████████████████| (done) 100%
NEW TRAIN STEP                                        
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |                                     
████████████████████████████████████████████████████████████████| (done) 100%
rulefit Model Build progress: |                       
█                                                     
██████████████████████

In [45]:
crossvals_list

[{'params': {'max_num_rules': 150, 'max_rule_length': 11, 'min_num_rules': 9},
  'cv_oos_0_mse': 0.842329501003499,
  'cv_oos_0_mae': 0.7093404546293126,
  'cv_oos_0_r2': 0.8688512669417406,
  'cv_train_0_mse': 0.5066164543262343,
  'cv_train_0_mae': 0.5924270876188149,
  'cv_train_0_r2': 0.922661858154533,
  'cv_oos_1_mse': 0.9856985622921004,
  'cv_oos_1_mae': 0.7392323943205656,
  'cv_oos_1_r2': 0.8495270049434086,
  'cv_train_1_mse': 0.4264697990240497,
  'cv_train_1_mae': 0.50307812580502,
  'cv_train_1_r2': 0.9335996498247041},
 {'params': {'max_num_rules': 310, 'max_rule_length': 14, 'min_num_rules': 9},
  'cv_oos_0_mse': 0.8193959065492602,
  'cv_oos_0_mae': 0.7046536427499756,
  'cv_oos_0_r2': 0.8724219739555187,
  'cv_train_0_mse': 0.49532530699016813,
  'cv_train_0_mae': 0.5808237223352385,
  'cv_train_0_r2': 0.9243855217797821,
  'cv_oos_1_mse': 0.9856958070003022,
  'cv_oos_1_mae': 0.7392365730918783,
  'cv_oos_1_r2': 0.8495274255557793,
  'cv_train_1_mse': 0.4264730317250

In [42]:
import pandas as pd

train_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\train_pKa_v4_features_2.2.csv', index_col=0)
test_df = pd.read_csv(r'C:\work\DrugDiscovery\main_git\XAI_Chem\data\updated_features\csv_for_rulefit\test_pKa_v4_features_2.2.csv', index_col=0)

train = h2o.H2OFrame(train_df[PKA_FEATURES + ['pKa', 'fold_id']])
test = h2o.H2OFrame(test_df[PKA_FEATURES + ['pKa']])

x = train.columns
x.remove('pKa')
x.remove('fold_id')
y = 'pKa'

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [44]:
from h2o.estimators import H2ORuleFitEstimator

rfit = H2ORuleFitEstimator(
                           max_rule_length=12,
                           max_num_rules=688,
                           min_rule_length=9,
                           seed=1,
                           model_type="rules")
rfit.train(training_frame=train, x=x, y=y, validation_frame=test)

rulefit Model Build progress: |

██████████████████████████████████████████████████| (done) 100%


,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,rule_ensemble_size,number_of_trees,number_of_internal_trees,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,gaussian,identity,Lasso (lambda = 0.02881 ),16557,106,1,16557.0,200.0,200.0,0.0,12.0,10.32,0.0,99.0,82.785
